# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Importing the dataset

In [2]:
dataset = pd.read_excel('Arrival_time.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_mass        576 non-null    float64
 1   Standoff_distance  576 non-null    int64  
 2   Time_of_arrival    576 non-null    float64
 3   Arrival_index      576 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 18.1 KB


In [3]:
y = dataset['Time_of_arrival']
X = dataset.drop(['Time_of_arrival','Arrival_index'], axis=1)
print(X.shape, y.shape)

(576, 2) (576,)


In [4]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

### Hyperparameter tuning 1 - layers, neurons, activation function

In [6]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as KR
import math
def FindLayerNodesLinear(first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ 2
    nodes = first_layer_nodes
    for i in range(1, 4):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [7]:
from tensorflow.keras.optimizers import Adam
def create_model(first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(first_layer_nodes, last_layer_nodes)
    for i in range(1, 4):
        if i==1:
            model.add(Dense(units = first_layer_nodes,  input_shape=(X_train.shape[1],), activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='linear'))
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['mae'])
    return model

##Wrap model into scikit-learn
model = KR(build_fn=create_model, epochs = 500, batch_size = 50)

C:\Users\z5379606\AppData\Local\Temp\ipykernel_79836\867054689.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model, epochs = 500, batch_size = 50)


In [8]:
activation_funcs = ['relu', 'softplus', 'leaky_relu'] 
param_grid = dict(first_layer_nodes = [250, 300, 350],
                  last_layer_nodes = [100, 120, 150],
                  activation_func = activation_funcs)
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
9/9 [==============================] - 1s 5ms/step - loss: 4.4085 - mae: 1.6969
Epoch 2/500
9/9 [==============================] - 0s 4ms/step - loss: 0.9576 - mae: 0.8218
Epoch 3/500
9/9 [==============================] - 0s 4ms/step - loss: 0.3828 - mae: 0.5228
Epoch 4/500
9/9 [==============================] - 0s 4ms/step - loss: 0.1437 - mae: 0.3605
Epoch 5/500
9/9 [==============================] - 0s 4ms/step - loss: 0.0462 - mae: 0.1955
Epoch 6/500
9/9 [==============================] - 0s 4ms/step - loss: 0.0129 - mae: 0.0995
Epoch 7/500
9/9 [==============================] - 0s 4ms/step - loss: 0.0050 - mae: 0.0603
Epoch 8/500
9/9 [==============================] - 0s 4ms/step - loss: 0.0026 - mae: 0.0431
Epoch 9/500
9/9 [==============================] - 0s 4ms/step - loss: 0.0016 - mae: 0.0331
Epoch 10/500
9/9 [==============================] - 0s 4ms/step - loss: 0.0011 - mae: 0.0251
Epoch 11/500
9/9 [==============================] - 0s 4ms/step - loss: 4.9150e

### Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [9]:
# Function to create model, required for KerasRegressor
def create_model(optimizer):
 # create model
 model = Sequential()
 model.add(Dense(units=350, input_shape=(X_train.shape[1],), activation='softplus'))
 model.add(Dense(units=250, activation='softplus'))
 model.add(Dense(units=150, activation='softplus'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [10]:
from scikeras.wrappers import KerasRegressor
# create model
model = KerasRegressor(model=create_model)

# define the grid search parameters
batch_size = [10,20,30]
epochs = [300, 400, 500]
optimizer = ['RMSprop', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, model__optimizer=optimizer, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/300
14/14 [==============================] - 1s 4ms/step - loss: 1.6365 - mae: 1.0614
Epoch 2/300
14/14 [==============================] - 0s 3ms/step - loss: 0.1606 - mae: 0.3455
Epoch 3/300
14/14 [==============================] - 0s 3ms/step - loss: 0.0454 - mae: 0.1877
Epoch 4/300
14/14 [==============================] - 0s 3ms/step - loss: 0.0079 - mae: 0.0771
Epoch 5/300
14/14 [==============================] - 0s 3ms/step - loss: 0.0029 - mae: 0.0471
Epoch 6/300
14/14 [==============================] - 0s 3ms/step - loss: 5.9563e-04 - mae: 0.0200
Epoch 7/300
14/14 [==============================] - 0s 3ms/step - loss: 2.3734e-04 - mae: 0.0122
Epoch 8/300
14/14 [==============================] - 0s 3ms/step - loss: 1.2143e-04 - mae: 0.0089
Epoch 9/300
14/14 [==============================] - 0s 3ms/step - loss: 8.2913e-05 - mae: 0.0076
Epoch 10/300
14/14 [==============================] - 0s 3ms/step - loss: 7.1991e-05 - mae: 0.0069
Epoch 11/300
14/14 [===================